<a href="https://colab.research.google.com/github/ddinesan/Manga/blob/master/Lecture_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quantum Fourier Transform Implementation







This is a worksheet prepared by Jing Song Du for Lecture 11 of the reading course "Introduction to Quantum Computer Programming" (AMATH 900/ AMATH 495/ QIC 895) at the University of Waterloo.

Course Webpage: https://sites.google.com/view/quantum-computer-programming

Text followed in the course: [Quantum Computing, An Applied Approach](https://www.springer.com/gp/book/9783030239213) by Jack D. Hidary (2019)


In [ ]:
# install cirq
!pip install cirq==0.5 --quiet

### What is a QFT?




Scott Aaronson Blog

QFT is a linear transformation (indeed a unitary transformation) that maps one vector of complex numbers to another vector of complex numbers

###Circuit Diagram

The construction of the QFT as a circuit follows a simple recursive form

>![QFT Circuit](https://upload.wikimedia.org/wikipedia/commons/6/61/Q_fourier_nqubits.png)


$H$ is the Hadamard gate, as usual. The Controlled-$R_j$ gates are phase gates similar to the Controlled-$Z$ gate. In fact, for us it will be useful to just think of them as fractional powers of Controlled-$Z$ gates:
$$
CR_j = CZ^{\large 1/2^{j-1}}
$$

### Quantum Fourier Transform as a Circuit

Let's define a generator which produces the QFT circuit. It should accept a list of qubits as input and `yield`s the gates to construct the QFT in the right order. A useful observation is that the the QFT circuit "repeats" smaller versions of itself as you move from left to right across the diagram.


```
  """Generator for the QFT on an arbitrary number of qubits. With four qubits
  the answer is
  ---H--@-------@--------@---------------------------------------------
        |       |        |
  ------@^0.5---+--------+---------H--@-------@------------------------
                |        |            |       |
  --------------@^0.25---+------------@^0.5---+---------H--@-----------
                         |                    |            |
  -----------------------@^0.125--------------@^0.25-------@^0.5---H---
  """
```


#### Solution

In [ ]:
import cirq
import numpy as np
import random

def make_qft(qubits):
  qubits = list(qubits)
  while len(qubits) > 0:
      q_head = qubits.pop(0)
      yield cirq.H(q_head)
      for i, qubit in enumerate(qubits):
          yield (cirq.CZ**(1/2**(i+1)))(qubit, q_head)

In [ ]:
num_qubits = 4
qubits = cirq.LineQubit.range(num_qubits)

qft = cirq.Circuit.from_ops(make_qft(qubits))
print(qft)

                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@───────────────────────────────────────
          │        │           │
1: ───────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                   │           │      │         │
2: ────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                               │                │          │
3: ────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘


### Quantum Fourier Transform as a Gate



For later convenience, it will be useful to encapsulate the QFT construction into a single gate. We can inherit from  `cirq.Gate` to define a gate which acts on an unspecified number of qubits, and then use the same strategy as for `make_qft` in the `_decompose_` method of the gate. Fill in the following code block to make a QFT gate.

In [ ]:
class QFT(cirq.Gate):
  """Gate for the Quantum Fourier Transformation
  """
  
  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits
    
  def _decompose_(self, qubits):
    qubits = list(qubits)
    while len(qubits) > 0:
        q_head = qubits.pop(0)
        yield cirq.H(q_head)
        for i, qubit in enumerate(qubits):
            yield (cirq.CZ**(1/2**(i+1)))(qubit, q_head)
            
  # How should the gate look in ASCII diagrams?          
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}'.format(i) for i in range(self.n_qubits))

#### Test the Circuit

We should confirm that the gate we've defined is actually doing the same thing as the `make_qft` function from before. We can do that with the following test:

In [ ]:
num_qubits = 4

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit.from_ops(QFT(num_qubits).on(*qubits))
print(circuit)

qft_test = cirq.Circuit.from_ops(make_qft(qubits))
print(qft_test)
np.testing.assert_allclose(cirq.unitary(qft_test), cirq.unitary(circuit))

0: ───QFT0───
      │
1: ───QFT1───
      │
2: ───QFT2───
      │
3: ───QFT3───
                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@───────────────────────────────────────
          │        │           │
1: ───────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                   │           │      │         │
2: ────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                               │                │          │
3: ────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘


In [ ]:
num_qubits = 4

qubits = cirq.LineQubit.range(num_qubits + 1)
circuit = cirq.Circuit.from_ops([QFT(num_qubits).on(*qubits[:4]), cirq.H(qubits[-1])])
print(circuit)

0: ───QFT0───
      │
1: ───QFT1───
      │
2: ───QFT2───
      │
3: ───QFT3───

4: ───H──────


#### Inverse QFT

We also want to implement the inverse QFT, which we'll do with a completely separate gate. Modify the `QFT` gate from above to create a `QFT_inv` gate:

Compared to the `QFT` code above, we just have to add in a few minus signs. You can convince yourself that this is the same as complex-conjugating the associated unitary matrix of the QFT, which gives us the inverse QFT.

In [ ]:
class QFT_inv(cirq.Gate):
  """Gate for the inverse Quantum Fourier Transformation
  """
  
  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits   
    
  def _decompose_(self, qubits):
    """Implements the inverse QFT on an arbitrary number of qubits. The circuit
    for num_qubits = 4 is given by
    ---H--@-------@--------@---------------------------------------------
          |       |        |
    ------@^-0.5--+--------+---------H--@-------@------------------------
                  |        |            |       |
    --------------@^-0.25--+------------@^-0.5--+---------H--@-----------
                           |                    |            |
    -----------------------@^-0.125-------------@^-0.25------@^-0.5--H---
    """
    qubits = list(qubits)
    while len(qubits) > 0:
        q_head = qubits.pop(0)
        yield cirq.H(q_head)
        for i, qubit in enumerate(qubits):
            yield (cirq.CZ**(-1/2**(i+1)))(qubit, q_head)
                      
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}^-1'.format(i) for i in range(self.n_qubits))

#### Solution

The QFT and QFT_inv circuits we have defined are not literal inverses of each other because the both reverse the order of the bits when going from input to output. We can explicitly see this in the following code block:

In [ ]:
num_qubits = 2

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit.from_ops(QFT(num_qubits).on(*qubits),
                                QFT_inv(num_qubits).on(*qubits))
print(circuit)
cirq.unitary(circuit).round(2)

0: ───QFT0───QFT0^-1───
      │      │
1: ───QFT1───QFT1^-1───


array([[1. +0.j , 0. +0.j , 0. +0.j , 0. +0.j ],
       [0. +0.j , 0.5+0.5j, 0. +0.j , 0.5-0.5j],
       [0. +0.j , 0.5+0.j , 0.5-0.5j, 0. +0.5j],
       [0. +0.j , 0. -0.5j, 0.5+0.5j, 0.5+0.j ]])

If `QFT` and `QFT_inv` were really inverses then we would have gotten the identity matrix here. There are a couple of ways to fix this. One is to change the implementations of these two gates in such a way that the outputs are "rightside-up." A different solution is to turn the qubits around in between acting with `QFT` and `QFT_inv`. In other words, we can insert the `QFT_inv` gate "upside-down" as follows:

In [ ]:
num_qubits = 2

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit.from_ops(QFT(num_qubits).on(*qubits),
                                QFT_inv(num_qubits).on(*qubits[::-1])) # qubit order reversed
print(circuit)
cirq.unitary(circuit)

0: ───QFT0───QFT1^-1───
      │      │
1: ───QFT1───QFT0^-1───


array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

We find the identity matrix, as desired (up to finite-precision numerical errors). Notice that the `QFT_inv` gate is upside-down relative to the `QFT` gate in the diagram. This is why we included the extra digits in the `wire_symobls`!

## Quantum Fourier Transform By Hidary

In many quantum computing systems there are constraints one which qubits can interact with each other. For example perhaps only nearest-neighbor qubits can interact. Then we cannot apply the standard QFT circuit described above.

In [ ]:
def _cz_and_swap(q0, q1, rot):
    yield cirq.CZ(q0, q1)**rot
    yield cirq.SWAP(q0,q1)

def generate_2x2_grid_qft_circuit():
    # Define a 2*2 square grid of qubits.
    a,b,c,d = [cirq.GridQubit(0, 0), cirq.GridQubit(0, 1),
               cirq.GridQubit(1, 0), cirq.GridQubit(1, 1)]

    circuit = cirq.Circuit.from_ops(
        cirq.H(a),
        _cz_and_swap(a, b, 0.5),
        _cz_and_swap(b, c, 0.25),
        _cz_and_swap(c, d, 0.125),
        cirq.H(a),
        _cz_and_swap(a, b, 0.5),
        _cz_and_swap(b, c, 0.25),
        cirq.H(a),
        _cz_and_swap(a, b, 0.5),
        cirq.H(a),
        strategy=cirq.InsertStrategy.EARLIEST
    )
    return circuit

# Modified QFT circuit includes SWAP operations fit for running on 
# a 2z2 grid of qubits with nearest-neightbour interactions
qft_circuit = generate_2x2_grid_qft_circuit()
print('Circuit:')
print(qft_circuit)
# Simulate and collect final_state
simulator = cirq.Simulator()
result = simulator.simulate(qft_circuit)
print()
print('FinalState')
print(np.around(result.final_state, 3))

Circuit:
(0, 0): ───H───@───────×───H────────────@─────────×───H────────────@───────×───H───
               │       │                │         │                │       │
(0, 1): ───────@^0.5───×───@────────×───@^0.5─────×───@────────×───@^0.5───×───────
                           │        │                 │        │
(1, 0): ───────────────────@^0.25───×───@─────────×───@^0.25───×───────────────────
                                        │         │
(1, 1): ────────────────────────────────@^(1/8)───×────────────────────────────────

FinalState
[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j
 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]


##Nearest Neighbour QFT Generator

In [ ]:
def cz_and_swap(q0, q1, rot):
    """Yields a controlled-RZ gate and SWAP gate on the input
    qubits."""
    yield cirq.CZ(q0, q1)**rot 
    yield cirq.SWAP(q0,q1)


def make_qft2(qubits):

    register_length = len(qubits)
    for i in range(0, register_length):
        yield cirq.H(qubits[0])
        for j in range(0, register_length - i - 1):
          yield cz_and_swap(qubits[j], qubits[j+1], 1/2**(j+1))

qubits = [cirq.GridQubit(0, 0), cirq.GridQubit(0, 1),
               cirq.GridQubit(1, 0), cirq.GridQubit(1, 1)]

# qubits = [cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), 
#                cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), 
#            cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]

qft = cirq.Circuit.from_ops(make_qft2(qubits))
print(qft)
print('FinalState')
print(np.around(result.final_state, 3))


(0, 0): ───H───@───────×───H────────────@─────────×───H────────────@───────×───H───
               │       │                │         │                │       │
(0, 1): ───────@^0.5───×───@────────×───@^0.5─────×───@────────×───@^0.5───×───────
                           │        │                 │        │
(1, 0): ───────────────────@^0.25───×───@─────────×───@^0.25───×───────────────────
                                        │         │
(1, 1): ────────────────────────────────@^(1/8)───×────────────────────────────────
FinalState
[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j
 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]


##Nearest Neighbour QFT Gate

In [ ]:
class QFT2(cirq.Gate):

  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits   

  def cz_and_swap(q0, q1, rot):
    """Yields a controlled-RZ gate and SWAP gate on the input
    qubits."""
    yield cirq.CZ(q0, q1)**rot 
    yield cirq.SWAP(q0,q1)
    
  def _decompose_(self, qubits):
    register_length = len(qubits)
    for i in range(0, register_length):
        yield cirq.H(qubits[0])
        for j in range(0, register_length - i - 1):
          yield cz_and_swap(qubits[j], qubits[j+1], 1/2**(j+1))
            
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}'.format(i) for i in range(self.n_qubits))

#Cirq's Default Implementation
https://cirq.readthedocs.io/en/stable/_modules/cirq/ops/fourier_transform.html#QFT

#References

https://www.scottaaronson.com/blog/?p=208

Stefan Leichanauer Cirq Algos Day 2

Hidary, Jack. (2019). Quantum Computing: An Applied Approach. 10.1007/978-3-030-23922-0. 

https://colab.research.google.com/drive/1X0H39CWQzx2uO9UGiokdseWsxt6ckxOw#scrollTo=lORoela1QICx

https://dkopczyk.quantee.co.uk/qft/

